# Introduction aux GLMs

Pour rappel, une simple régression linéaire a le format suivant:

$y_i = \beta_0 + \beta_1 x_1 + \epsilon_i$

Ce type de modèle est très simple mais souvent suffisant. Par contre, dans certaines situation, un modèle linéaire simple n'est pas approprié, notamment:

- si les valeurs acceptables pour la variable dépendante sont limitées (p.ex. des chiffres entiers seulement, ou des catégories)
- si la variance de la variable dépendante dépend de la moyenne

Les modèles linéaires généralisés (Generalized Linear Models, GLM) permettent de faire face à ces situations.

Un modèle linéaire généralisé est composé de 3 éléments:

1. Une **prediction linéaire** du style
$$\eta_1 = \beta_0 + \beta_1 x_{1i} + \dots + \beta_p x_{pi}$$

2. Une **fonction de lien** (_«link function»_) qui détermine comment la prédiction $\eta_i$ est liée à l'espérence $E(Y_i)$ (la moyenne $\mu_i$):
$$g(\mu_i) = \eta_i$$

3. une **fonction de variance** qui décrit comment la variance de $Y_i$ est liée à la moyenne:
    $$var(Y_i) = \phi V(\mu)$$
    ($\phi$ est une constante appelée le paramètre de dispersion)

De cette manière, à travers la fonction de lien, nous pouvons avoir des régressions qui ne sont pas linéaires, mais où le modèle est toujours une combinaison linéaire de paramètres.

Nous pouvons en principe obtenir un modèle linéaire simple (**modèle linéaire général**) _(attention: général ≠ généralisé)_ avec la fonction de lien $g(\mu_i) = \mu_i$ et la fonction de variance $V(\mu_i) = 1$.

Avec une fonction de lien $g(\mu) = log(\mu)$, nous pouvons faire la régression linéaire sur le logarithme de la variable dépendante; nous parlons alors d'un **modèle log-linéaire**.

Si la fonction de lien prend la forme $g(\mu) = log(\frac{\mu}{1-\mu})$, nous parlons d'un **modèle logit** (régression logistique), qui permet de modéliser la probabilité binomiale.

## Types de GLM

Dans la pratique, nous allons rencontrer les situations suivantes qui permettent l'application d'un modèle linéaire généralisé:

| | Type de variable | Distribution | Type de GLM | Fonction de lien |
|-|------------------|--------------|-------------|------------------|
|1| Variable continue suivant une loi normale | Normale | Modèle linéaire simple | Identité: $g(\mu) = \mu$ |
|2| Variable représentant un comptage (nombres entiers positifs) | Poisson | Modèle log-linéaire | Log: $g(\mu) = log(\mu)$ |
|3| Pourcentages, probabilités (2 classes) | Binomiale | Régression logistique | Logit: $g(\mu) = log(\frac{\mu}{1-\mu})$ |
|4| Probabilités ($k$ classes) | Multinomiale | Régression logiqtique | Logit: $g(\mu) = log(\frac{\mu}{1-\mu})$ |
|5| Variable continue positive (p.ex. des durées) | Gamma | Modèle Gamma avec fonction de lien inverse | Inverse: $g(\mu) = \frac{1}{\mu}$ |

Généralement, il suffit presque de savoir quel type de variable nous avons, et de choisir le modèle en fonction...

Le cas du **GLM** en «mode» **modèle linéaire simple** est identique à la régression linéaire ordinaire que nous avons déjà vu, à part pour le mode de calcul qui est différent (méthodes des moindres carrés ordinaires pour la régression classique, et le maximum de vraisemblance dans le cas du GLM), ce qui a des implications notamment sur l'interprétation (calcul du $R^2$ ou non). Ici, nous ne traitons donc pas ce cas particulier des GLM.

Nous avons également déjà traité le cas du GLM en «mode» **régression logistique**.

Le cas de la **régression logistique multinomiale** n'est pas couvert par la fonction `glm` dans R. Nous allons utiliser une approche alternative qui est décrite dans [3-logit-multinomiale.ipynb](3-logit-multinomiale.ipynb).

### Régression de Poisson

La régression de Poisson est utilisée pour les données de comptages, qui sont des nombres entiers non-négatifs. Nous pouvons utiliser ce type de régression par exemple pour estimer des fréquences, ou les changements dans la fréquentation d'un lieu suite à une intervention (construction de nouveaux logements ou de magasins, etc.).

Pour utiliser une régression de Poisson, la variable dépendante doit suivre une distribution de Poisson. La forme de cette distribution peut varier en fonction d'un paramètre $\lambda$ et prendre les formes suivantes:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/16/Poisson_pmf.svg/650px-Poisson_pmf.svg.png" style="max-width: 400px;" />  
<p style="text-align: center; font-size: 80%"><i>Source: https://commons.wikimedia.org/wiki/File:Poisson_pmf.svg</i></p>

Dans R, le calcul se fait évidemment avec la fonction `glm`, où seul le paramètre `family` change:

```R
glm(..., family=poisson(link="log"), data=...)
```

En raison du type de données de la variable dépendante, des nombres entiers, la régression de Poisson est parfois utilisé pour estimer les flux entres régions, notamment à travers les **modèles d'interaction spatiale**, dont le modèle gravitaire est le représentant le plus simple.

Rappelons-nous qu'un modèle d'interaction spatiale peut prendre la forme suivante:

$$T_{ij} = k \cdot P_i^\lambda \cdot P_j^\alpha \cdot d_{ij}^{-\beta}$$

où

- $T_{ij}$ est le nombre de flux entre régions à estimer
- $P_i$ est la population à l'origine
- $P_j$ est la population à la destination
- $d_{ij}$ est la distance ou le coût entre régions
- $k$, $\lambda$, $\alpha$ et $\beta$ sont des paramètres à trouver avec la calibration

Ce modèle peut être transformé en modèle linéaire en prenant le logarithme:

$$\ln T_{ij} = k + \lambda \ln P_i + \alpha \ln P_j - \beta \ln d_{ij}$$

ce qui est maintenant un modèle linéaire que nous pouvons calibrer avec la méthode GLM.

### Régression Gamma

Si la variable dépendante suit une distribution Gamma, on peut utiliser un modèle GLM en mode «régression Gamma». La forme de la distribution Gamma peut varier en fonction de 2 paramètres, $k$ et $\theta$:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Gamma_distribution_pdf.svg/650px-Gamma_distribution_pdf.svg.png" style="max-width: 450px;" />
<p style="font-size: 80%; text-align: center;"><i>Source: https://commons.wikimedia.org/wiki/File:Gamma_distribution_pdf.svg</i></p>

La distribution Gamma ne permet donc pas de valeurs négatives. Souvent, des données quantitatives non-négatives suivent une distribution Gamma. Un exemple où la régression Gamma a déjà été utilisée est l'estimation la quantité de pluie lors d'une journée pluvieuse.

Dans R, la régression Gamma ne se distingue que par le paramètre `family` dans la fonction `glm`:

```R
glm(..., family=Gamma(link="inverse"), data=...)
```